<a href="https://colab.research.google.com/github/nicklausmillican/StatisticalRethinkingIISolutions/blob/main/StatisticalRethinkingSolutions2_Ch7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7

In [ ]:
install.packages(c("coda","mvtnorm","devtools","loo","dagitty"))
devtools::install_github("rmcelreath/rethinking@slim")

In [ ]:
library(rethinking)

## Easy

### 7E1
#### Question
State the three motivating criteria that define information entropy. Try to express each in your  own words.

#### Answer
Entropy is a measure of uncertainty; information resolves that uncertainty (e.g., with evidence).  So entropy *feels* a bit like probability in that both deal with levels of (un)certainty.  It shouldn't be surprising, then, that entropy H is a function of probability.

$$H(X) = \sum_{i=1}^n p(X=x_i) \times log_b(\frac{1}{p(X=x_i)})$$
$$= - \sum_{i=1}^n p(X=x_i) \times log_b(p(X=x_i))$$
$$= -E[log_b(p(X=x_i))]$$

How to interpret?  Think of $log_b(\frac{1}{p(X=x_i)})$ as "surprise": the greater the probability $p(X=x_i)$, the less surprising it is when that event occurs; conversely, the less probable an event $p(X=x_i)$, the more surprising it is when it occurs.  Next, we *weight* the probability of each event $p(X=x_i)$ by its probability of occuring $p(X=x_i)$ and sum over each event $X=x_i$.  This gives us a *weighted average* of surprise for $X$.  We actually use the $log$ of $\frac{1}{p(X=x_i)}$ in order to achieve a few desirable traits for our measure of uncertainty.

From p. 205:
1.   **Continuity:** Just as we want our (un)certainty to be able to slide smoothly from completely uncertain to completely certain, we want informational entropy to do the same.  Probability accomplishes this and, as a function of probability, the formula for $H$ permits the same.
2.   **Proportionality:** All else being equal, more potential outcomes for $X$ should increase our uncertainty about $X$.  You can see how the formula for $H$ accomplishes this.  If instead of being, say, 3 values for $X$ there are 4, $H$ becomes the sum of 4 terms instead of only 3; plus, the additional 4th value for $X$ means that the probability of each possible value has probably decreased--thus increasing the surprise for each.  All told, $H$ increases.
3.   **Additivity:** If we have two events about which we are uncertain, it is desirable to say that our total uncertainty is the sum of those events.  Similarly for more events.  The formula for $H$ accomplishes this by the $\sum$ operator.  We sum the surprise, weighted by its probability of happening, of each possible event.

Additionally, there are some other important features of information entropy no mentioned in the book:
4.   **Non-Negativity:** We cannot be negatively uncertain about an event; we can only be completely certain--which is 0 uncertainty.  The formula for $H$ accomplishes this by its use of probabilities: probabilities also cannot be negative.
5.   **Maximal Value:** Just as there is a minimal entropy (0), there is a maximal entropy.  Maximal entropy occurs when each possible value are equally likely.  You can see this in the formula for $H$: if the probability of any event becomes greater than $\frac{1}{n}$, then the probabilities of other events must decrease...increasing their surprise.  This results in less overall entropy.

> This chapter addresses a concept called "MaxEnt" or "maximal entropy".  This is a similar concept, but it refers to the maximal entropy of a variable under some set of contraints.

There are many other features of informational entropy that we could list.  But we'll stop here.

If you're interested in a deeper but still-understandable resource on information theory, I suggest [Probability and Information: An Integrated Approach 2nd Edition](https://www.amazon.com/Probability-Information-Integrated-David-Applebaum/dp/0521899044) and [Information Theory: A Tutorial Introduction (2nd Edition)](https://www.amazon.com/Information-Theory-Tutorial-Introduction-2nd/dp/1739672704/ref=sr_1_1?crid=QB4WC07L2QV2&dib=eyJ2IjoiMSJ9.HdyNIMnteFZLf7Ghuh6b4KpfMMBis3Cg2Cn4pOhcL08uhNjOVjY5qqMtASytMwiCDZNo8atQ_BvoUOLSeLvzJMSkRrLHGJNdYa3VnLzWgodcgfMRGbJkHt5VFKslyyzX4JYNra34ExCHrvPo7sXCCkIN3NFpJom82G6K_FzCkaU-mOKz-PkQ3CnNhjkBzmYSdIkVdNUmWSE-Di1EjxG_4rWfCra_68Z8zvOI4yM1ub0uKR_QGV0xFv66L61PHlPS25GTH9hCAOTv5q0nezWOtHUX9fO6YEyj17fuskNrV5M.dpOwik_DNtNOYUJyMa8EC825qBdeL0uQJYgazlfU7l0&dib_tag=se&keywords=information+theory&qid=1709823456&s=books&sprefix=information+theory%2Cstripbooks%2C151&sr=1-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1).

### 7E2-7E4
#### Questions
2.   Suppose a coin is weighted such that, when it is tossed and lands on a table, it comes up heads 70% of the time. What is the entropy of this coin?
3.   Suppose a four-sided die is loaded such that, when tossed onto a table, it shows “1” 20%, “2”  25%, “3” 25%, and “4” 30% of the time. What is the entropy of this die?  
4.   Suppose another four-sided die is loaded such that it never shows “4”. The other three sides show equally often. What is the entropy of this die?

#### Answers
We'll group these answers, since they all follow the same process.  We just need to define the probabilities given in the question, then implement our formula for entropy.

For each, I'll calculate entropy in a few different ways, and also compare it to the maximal entropy for that situation described in the question.

Finally, I simulate data according to the situation described in the problem, and then calculate the empirical entropy from that generated data, both manually and with the `entropy` package.

In [42]:
install.packages("entropy")
library(entropy)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



##### 7E2
***Suppose a coin is weighted such that, when it is tossed and lands on a table, it comes up heads 70% of the time. What is the entropy of this coin?***

In [43]:
p <- c(0.7, 1-0.7)
maxP <- c(0.5, 1-0.5)

(H1 <- -sum(p*log(p)))
(H2 <- sum(p*log(1/p)))

(maxH1 <- -sum(maxP*log(maxP)))
(maxH2 <- sum(maxP*log(1/maxP)))

[1] 0.6108643

[1] 0.6108643

[1] 0.6931472

[1] 0.6931472

In [46]:
outcomes <- c("heads", "tails")
d <- sample(x=outcomes, size=100, replace=TRUE, prob=p)

(empiricalP <- c(sum(d=="heads")/length(d), sum(d=="tails")/length(d)))

(empiricalH <- -sum(empiricalP*log(empiricalP)))
entropy(d) # entropy package

[1] 0.62 0.38

[1] 0.6640641

[1] 0.6640641

##### 7E3
***Suppose a four-sided die is loaded such that, when tossed onto a table, it shows “1” 20%, “2” 25%, “3” 25%, and “4” 30% of the time. What is the entropy of this die?***

In [47]:
p <- c(0.2, 0.25, 0.25, 0.3)
maxP <- c(0.25, 0.25, 0.25, 0.25)

(H1 <- -sum(p*log(p)))
(H2 <- sum(p*log(1/p)))

(maxH1 <- -sum(maxP*log(maxP)))
(maxH2 <- sum(maxP*log(1/maxP)))

[1] 1.376227

[1] 1.376227

[1] 1.386294

[1] 1.386294

In [48]:
outcomes <- c("1", "2", "3", "4")
d <- sample(x=outcomes, size=100, replace=TRUE, prob=p)

(empiricalP <- table(d)/length(d))

(empiricalH <- -sum(empiricalP*log(empiricalP)))
entropy(d) # entropy package

d
   1    2    3    4 
0.23 0.26 0.23 0.28 

[1] 1.38272

[1] 1.38272

##### 7E5
***Suppose another four-sided die is loaded such that it never shows “4”. The other three sides show equally often. What is the entropy of this die?***

In [49]:
p <- c(1/3, 1/3, 1/3)
maxP <- c(0.25, 0.25, 0.25, 0.25)

(H1 <- -sum(p*log(p)))
(H2 <- sum(p*log(1/p)))

(maxH1 <- -sum(maxP*log(maxP)))
(maxH2 <- sum(maxP*log(1/maxP)))

[1] 1.098612

[1] 1.098612

[1] 1.386294

[1] 1.386294

In [50]:
outcomes <- c("1", "2", "3")
d <- sample(x=outcomes, size=100, replace=TRUE, prob=p)

(empiricalP <- table(d)/length(d))

(empiricalH <- -sum(empiricalP*log(empiricalP)))
entropy(d) # entropy package

d
   1    2    3 
0.31 0.33 0.36 

[1] 1.09672

[1] 1.09672